In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -48.4814,
	"longitude": -72.5891,
	"hourly": ["temperature_2m","relative_humidity_2m", "dew_point_2m", "rain", "snowfall", "pressure_msl", "surface_pressure", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "vapour_pressure_deficit", "wind_gusts_10m", "soil_moisture_0_to_7cm"],
	"start_date": "2024-08-08",
	"end_date": "2024-08-08"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(4).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(5).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(6).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(7).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(8).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(9).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(10).ValuesAsNumpy()
hourly_soil_moisture_0_to_1cm = hourly.Variables(11).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_1cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates -48.375°N -72.625°E
Elevation 257.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                        date  temperature_2m  relative_humidity_2m  \
0  2024-08-08 00:00:00+00:00            2.27                  2.27   
1  2024-08-08 01:00:00+00:00            2.27                  2.27   
2  2024-08-08 02:00:00+00:00            2.07                  2.07   
3  2024-08-08 03:00:00+00:00            1.82                  1.82   
4  2024-08-08 04:00:00+00:00            1.57                  1.57   
5  2024-08-08 05:00:00+00:00            1.57                  1.57   
6  2024-08-08 06:00:00+00:00            0.77                  0.77   
7  2024-08-08 07:00:00+00:00            0.67                  0.67   
8  2024-08-08 08:00:00+00:00            0.67                  0.67   
9  2024-08-08 09:00:00+00:00            0.87                  0.87   
10 2024-08-08 10:00:00+00:00            1.07                  1.07   
11 2024-08-08 11:00:00+00:00            1.42          

In [2]:
hourly_dataframe["hour"] = hourly_dataframe["date"].dt.hour
hourly_dataframe["period"] = hourly_dataframe["hour"].apply(lambda x: 1 if x <= 8 else (2 if x <= 16 else 3))
hourly_dataframe.head()


,date,temperature_2m,relative_humidity_2m,dew_point_2m,rain,snowfall,pressure_msl,surface_pressure,cloud_cover_low,cloud_cover_mid,cloud_cover_high,vapour_pressure_deficit,wind_gusts_10m,soil_moisture_0_to_7cm,hour,period
0,2024-08-08 00:00:00+00:00,2.27,2.27,77.0,-1.347583,0.0,0.0,1027.300049,995.164307,38.0,0.0,43.0,0.165963,11.520000,0,1
1,2024-08-08 01:00:00+00:00,2.27,2.27,77.0,-1.347583,0.0,0.0,1026.500000,994.389282,44.0,0.0,2.0,0.165963,10.799999,1,1
2,2024-08-08 02:00:00+00:00,2.07,2.07,77.0,-1.541730,0.0,0.0,1026.300049,994.172729,36.0,4.0,0.0,0.163613,12.240000,2,1
3,2024-08-08 03:00:00+00:00,1.82,1.82,77.0,-1.784420,0.0,0.0,1025.599976,993.465698,30.0,43.0,65.0,0.160717,11.879999,3,1
4,2024-08-08 04:00:00+00:00,1.57,1.57,78.0,-1.852015,0.0,0.0,1025.199951,993.049744,14.0,100.0,100.0,0.151004,12.240000,4,1


In [3]:
hourly_dataframe["date"]= hourly_dataframe["date"].dt.date

In [4]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     24 non-null     object 
 1   temperature_2m           24 non-null     float32
 2   relative_humidity_2m     24 non-null     float32
 3   dew_point_2m             24 non-null     float32
 4   rain                     24 non-null     float32
 5   snowfall                 24 non-null     float32
 6   pressure_msl             24 non-null     float32
 7   surface_pressure         24 non-null     float32
 8   cloud_cover_low          24 non-null     float32
 9   cloud_cover_mid          24 non-null     float32
 10  cloud_cover_high         24 non-null     float32
 11  vapour_pressure_deficit  24 non-null     float32
 12  wind_gusts_10m           24 non-null     float32
 13  soil_moisture_0_to_7cm   24 non-null     float32
 14  hour                     24 

In [5]:
df_period = hourly_dataframe.groupby(['date', 'period']).agg({"temperature_2m":'mean',
'relative_humidity_2m':'mean',
'dew_point_2m':'mean',
'rain':'mean',
'snowfall':'mean',
'pressure_msl':'mean',
'surface_pressure':'mean',
'cloud_cover_low':'mean',
'cloud_cover_mid':'mean',
'cloud_cover_high':'mean',
'vapour_pressure_deficit':'mean',
'wind_gusts_10m':'mean',
'soil_moisture_0_to_7cm':'mean'})

In [6]:
df_period.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3 entries, (datetime.date(2024, 8, 8), 1) to (datetime.date(2024, 8, 8), 3)
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   temperature_2m           3 non-null      float32
 1   relative_humidity_2m     3 non-null      float32
 2   dew_point_2m             3 non-null      float32
 3   rain                     3 non-null      float32
 4   snowfall                 3 non-null      float32
 5   pressure_msl             3 non-null      float32
 6   surface_pressure         3 non-null      float32
 7   cloud_cover_low          3 non-null      float32
 8   cloud_cover_mid          3 non-null      float32
 9   cloud_cover_high         3 non-null      float32
 10  vapour_pressure_deficit  3 non-null      float32
 11  wind_gusts_10m           3 non-null      float32
 12  soil_moisture_0_to_7cm   3 non-null      float32
dtypes: float32(13)
memory usag

In [7]:
df_period.head(24)

temperature_2m  relative_humidity_2m  dew_point_2m  \
date       period                                                       
2024-08-08 1             1.520000              1.520000      78.00000   
           2             1.707500              1.707500      83.50000   
           3             6.262857              6.262857      79.85714   

                       rain  snowfall  pressure_msl  surface_pressure  \
date       period                                                       
2024-08-08 1      -1.902466  0.000000           0.0       1025.022217   
           2      -0.829375  0.100000           0.0       1020.337524   
           3       3.025180  0.085714           0.0       1014.957153   

                   cloud_cover_low  cloud_cover_mid  cloud_cover_high  \
date       period                                                       
2024-08-08 1            992.871765        19.000000         43.888889   
           2            988.354980        76.750000         59.375000   
           3            983.652466        79.428574         51.142857   

                   vapour_pressure_deficit  wind_gusts_10m  \
date       period                                            
2024-08-08 1                     54.777779        0.150850   
           2                     25.250000        0.112784   
           3                      0.000000        0.194092   

                   soil_moisture_0_to_7cm  
date       period                          
2024-08-08 1                    11.559999  
           2                    15.254999  
           3                    19.491428

In [8]:
from pickle import load
import streamlit as st
model = load(open("model.sav", "rb"))



In [9]:
prediccion = model.predict(df_period)
print(f'Predicción para mañana: {prediccion}')

Predicción para mañana: [0.        0.        3.0232738]


In [10]:
prediccion[2]

3.0232738041877747

In [11]:
period_dict = {
    "0": "Mañana",
    "1": "Tarde",
    "2": "Noche"
}

periodos = ['Manana', 'Tarde', 'Noche']

In [14]:
#creo un df con las tres columnas de period_dict como nombre de columna
df = pd.DataFrame()

# creamos las columnas
df['Mañana'] = None
df['Tarde'] = None
df['Noche'] = None

df.loc['Prediccion'] = prediccion

In [15]:
df

,Mañana,Tarde,Noche
Prediccion,0.0,0.0,3.023274


In [17]:
from datetime import date
date.today().isoformat()

'2024-08-09'